In [107]:
import pandas as pd


db = pd.read_csv("/Users/home/Dropbox/Databases/fifa-world-cup/WorldCupMatches.csv")
db.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4572 entries, 0 to 4571
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Year                  852 non-null    float64
 1   Datetime              852 non-null    object 
 2   Stage                 852 non-null    object 
 3   Stadium               852 non-null    object 
 4   City                  852 non-null    object 
 5   Home Team Name        852 non-null    object 
 6   Home Team Goals       852 non-null    float64
 7   Away Team Goals       852 non-null    float64
 8   Away Team Name        852 non-null    object 
 9   Win conditions        852 non-null    object 
 10  Attendance            850 non-null    float64
 11  Half-time Home Goals  852 non-null    float64
 12  Half-time Away Goals  852 non-null    float64
 13  Referee               852 non-null    object 
 14  Assistant 1           852 non-null    object 
 15  Assistant 2          

In [108]:
db = db[['Year', 'Home Team Name', 'Home Team Goals',
         'Away Team Goals', 'Away Team Name']]


# I don't this code anymore because I found a data base that has the information 
# I will still leave it here
#db.describe()
#db['Total_goals'] = db['Home Team Goals'] + db['Away Team Goals']
#db.groupby('Year')['Total_goals'].mean() # Aggregate by mean
#db.groupby('Year')['Total_goals'].sum() # Aggregate get goals
#db.groupby('Year')['Total_goals'].count() # Aggregate number of games
#db0 = (db.groupby(['Year'])['Total_goals']
 #        .agg([('average_goals','mean'),('total_goals','sum'), ('total_games','count')])
 #        .reset_index())
#db0

In [109]:
#Aggregated information from world cups
db3 = pd.read_csv("/Users/home/Dropbox/Databases/fifa-world-cup/WorldCups.csv")
# Tranforms the attendance into an integer
db3['Attendance']=db3['Attendance'].replace('\.','',regex=True).astype(int)


In [110]:
# World cup 2018

db2 = pd.read_csv("/Users/home/Dropbox/Databases/fifa-world-cup/Cup.Russia.Matches.csv")

# Order the data to make all aggregate data
db2 = db2[['Home Team',
           'Home Team Goals',  'Away Team Goals',
           'Away Team', 'Total Goals', 'Attendance']]

db2['Year'] = 2018
db2['Country'] = 'Russia'
db2['GoalsScored'] = db2['Total Goals'].sum()

# Number of  Qualified teams

# Extract useful columns
db2_1 = db2['Home Team']
db2_2 = db2['Away Team']

# Rename columns to merge
db2_1.columns = ['Teams']
db2_2.columns = ['Teams']

# Row binding
data_merging = [db2_1, db2_2]
db2_12 = pd.concat(data_merging)

# Extract only unique cases
number = len(db2_12.unique().tolist())
db2['Qualifiedteams'] =  number

# Calculate total attendance and number of matches playes
db2['MatchesPlayed'] = db2['Home Team'].count()
db2['Total_attendance'] = db2['Attendance'].sum()

# Filter columns
db2_aggregation_var = db2[['Year', 'Country', 'GoalsScored',
                           'Qualifiedteams', 'MatchesPlayed',
                           'Total_attendance']]
# Create missing columns
db2_aggregation_var['Winner'] = 'France'
db2_aggregation_var['Runners-Up'] = 'Croatia' 
db2_aggregation_var['Third'] = 'Belgium'                    
db2_aggregation_var['Fourth'] = 'England' 

# Re organize columns
db2_aggregation_var = db2_aggregation_var[['Year', 'Country',
                                           'Winner', 'Runners-Up',
                                           'Third', 'Fourth', 
                                           'GoalsScored', 'Qualifiedteams',
                                           'MatchesPlayed', 'Total_attendance']]                    

# Rename columns to bind
db2_aggregation_var.columns =['Year', 'Country',
                                           'Winner', 'Runners-Up',
                                           'Third', 'Fourth', 
                                           'GoalsScored', 'QualifiedTeams',
                                           'MatchesPlayed', 'Attendance']                   

# Extract single row
db2_aggregation_var = db2_aggregation_var[0:1]
# Row binding
wc_row_bind = [db3, db2_aggregation_var]
WC_genera_final = pd.concat(wc_row_bind)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [119]:
# Create a variable of Year for Russia
db2['Year'] = 2018
# Re organize the variables
db2_matches = db2[['Year','Home Team', 'Home Team Goals', 'Away Team Goals',
                   'Away Team']]


# Rename the variables
db2_matches.columns = ['Year', 'Home Team Name', 'Home Team Goals', 
                      'Away Team Goals', 'Away Team Name']

matches_bind = [db,db2_matches]
WC_matches_final = pd.concat(matches_bind)
WC_matches_final

,Year,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name
0,1930.0,France,4.0,1.0,Mexico
1,1930.0,USA,3.0,0.0,Belgium
2,1930.0,Yugoslavia,2.0,1.0,Brazil
3,1930.0,Romania,3.0,1.0,Peru
4,1930.0,Argentina,1.0,0.0,France
...,...,...,...,...,...
59,2018.0,Russia,2.0,2.0,Croatia
60,2018.0,France,1.0,0.0,Belgium
61,2018.0,Croatia,2.0,1.0,England
62,2018.0,Belgium,2.0,0.0,England


In [ ]:
# Merge data sets

#db_final = pd.merge(db, db2, how='inner', left_on='Year', right_on='Year')
#db_final.info()